In [2]:
import numpy as np

import pandas as pd
pd.set_option('display.max_colwidth', 512)


from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.metrics import roc_auc_score
import pickle

### clean data

In [3]:
### build jc train

features = ["toxic","severe_toxic","obscene","threat","insult","identity_hate"]

jc_df = pd.read_csv("../processed/jc.csv")

jc_df.head()

,id,text,toxic,severe_toxic,obscene,threat,insult,identity_hate,layer,toxic_flag,group_id
0,0000997932d777bf,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0,0,0,0,0,0,train,0,1150
1,000103f0d9cfb60f,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0,0,0,0,0,0,train,0,1151
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0,0,0,0,0,0,train,0,1152
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be ...",0,0,0,0,0,0,train,0,1153
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember what page that's on?",0,0,0,0,0,0,train,0,1154


In [4]:
g = jc_df.groupby(["group_id"]).agg({"id":"count","toxic":"std","severe_toxic":"std","obscene":"std","threat":"std","insult":"std","identity_hate":"std"}).fillna(0)
g["std"] = g [features].sum(axis=1)
g.query("std != 0").sort_values(by="id", ascending = False)

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate,std
group_id,,,,,,,,
1139,121,0.090909,0.000000,0.128028,0.0,0.090909,0.000000,0.309846
788,8,0.000000,0.000000,0.517549,0.0,0.534522,0.000000,1.052072
481,4,0.577350,0.000000,0.000000,0.0,0.000000,0.000000,0.577350
651,4,0.000000,0.500000,0.000000,0.0,0.000000,0.000000,0.500000
526,4,0.577350,0.000000,0.000000,0.0,0.000000,0.000000,0.577350
...,...,...,...,...,...,...,...,...
443,2,0.000000,0.707107,0.000000,0.0,0.000000,0.000000,0.707107
444,2,0.000000,0.000000,0.000000,0.0,0.000000,0.707107,0.707107
447,2,0.707107,0.000000,0.000000,0.0,0.000000,0.000000,0.707107


In [5]:
jc_df.query("group_id == 1139").sort_values(by="toxic_flag", ascending=False)

,id,text,toxic,severe_toxic,obscene,threat,insult,identity_hate,layer,toxic_flag,group_id
152180,890db12df368a8a0,"Welcome Faggot!\n\nWelcome!\n\nHello, , and welcome to Wikipedia! Thank you for your contributions. I hope you like the place and decide to stay. Here are a few good links for newcomers:\nThe five pillars of Wikipedia\nHow to edit a page\nHelp pages\nTutorial\nHow to write a great article\nManual of Style\nI hope you enjoy editing here and being a Wikipedian! Please sign your name on talk pages using four tildes (~~~~); this will automatically produce your name and the date. If you need help, check out ...",1,0,1,0,1,0,train,3,1139
34184,5b3d61928be86ca8,"WELCOME, BITCH\n\nWelcome!\n\nHello, , and welcome to Wikipedia! Thank you for your contributions. I hope you like the place and decide to stay. Here are a few good links for newcomers:\nThe five pillars of Wikipedia\nHow to edit a page\nHelp pages\nTutorial\nHow to write a great article\nManual of Style\nI hope you enjoy editing here and being a Wikipedian! Please sign your name on talk pages using four tildes (~~~~); this will automatically produce your name and the date. If you need help, check out W...",0,0,1,0,0,0,train,1,1139
134310,ce3014268429336f,"Welcome!\n\nHello, , and welcome to Wikipedia! Thank you for your contributions. I hope you like the place and decide to stay. Here are a few good links for newcomers:\nThe five pillars of Wikipedia\nHow to edit a page\nHelp pages\nTutorial\nHow to write a great article\nManual of Style\nI hope you enjoy editing here and being a Wikipedian! Please sign your name on talk pages using four tildes (~~~~); this will automatically produce your name and the date. If you have any questions, check out Wikipedia:...",0,0,0,0,0,0,train,0,1139
144821,12f4422cf36ed980,"Welcome!\n\nHello, , and welcome to Wikipedia! Thank you for your contributions. I hope you like the place and decide to stay. Here are some pages that you might find helpful:\nThe five pillars of Wikipedia\nHow to edit a page\nHelp pages\nTutorial\nHow to write a great article\nManual of Style\nI hope you enjoy editing here and being a Wikipedian! Please sign your name on talk pages using four tildes (~~~~); this will automatically produce your name and the date. If you need help, check out Wikipedia:Q...",0,0,0,0,0,0,train,0,1139
144033,06e511d56140e690,"Welcome!\n\nHello, , and welcome to Wikipedia! Thank you for your contributions. I hope you like the place and decide to stay. Here are some pages that you might find helpful:\nThe five pillars of Wikipedia\nHow to edit a page\nHelp pages\nTutorial\nHow to write a great article\nManual of Style\nI hope you enjoy editing here and being a Wikipedian! Please sign your name on talk pages using four tildes (~~~~); this will automatically produce your name and the date. If you need help, check out Wikipedia:Q...",0,0,0,0,0,0,train,0,1139
...,...,...,...,...,...,...,...,...,...,...,...
60584,a22f92cc4b800e6e,"Welcome!\n\nHello, , and welcome to Wikipedia! Thank you for your contributions. I hope you like the place and decide to stay. Here are some pages that you might find helpful:\nThe five pillars of Wikipedia\nTutorial\nHow to edit a page\nHow to write a great article\nManual of Style\nI hope you enjoy editing here and being a Wikipedian! Please sign your name on talk pages using four tildes ~~~~; this will insert your username and the date. If you need help, check out Wikipedia:Questions, ask me on , or ...",0,0,0,0,0,0,train,0,1139
58387,9c4e67ec1d05410c,"Welcome!\n\nHello, , and welcome to Wikipedia! Thank you for your contributions. I hope you like the place and decide to stay. Here are a few good links for newcomers:\nThe five pillars of Wikipedia\nHow to edit a page\nHelp pages\nTutorial\nHow to write a great article\nManual of Style\nI hope you enjoy editing here and being a Wikipedian! Please sign your name on talk pages using four tildes (~~~~); this will automatically produce your name and the date. I

In [6]:
jc_df.loc[152180, "group_id"] = jc_df["group_id"].max()
jc_df.loc[34184, "group_id"] = jc_df["group_id"].max()


In [7]:
jc_df.query("group_id == 1139").sort_values(by="toxic_flag", ascending=False)

,id,text,toxic,severe_toxic,obscene,threat,insult,identity_hate,layer,toxic_flag,group_id
630,01ae8496770c49db,"Welcome!\n\nHello, , and welcome to Wikipedia! Thank you for your contributions. I hope you like the place and decide to stay. Here are some pages that you might find helpful:\nThe five pillars of Wikipedia\nHow to edit a page\nHelp pages\nTutorial\nHow to write a great article\nManual of Style\nI hope you enjoy editing here and being a Wikipedian! Please sign your name on talk pages using four tildes (~~~~); this will automatically produce your name and the date. If you need help, check out Wikipedia:Q...",0,0,0,0,0,0,train,0,1139
134098,cd2c8bce591bd1a9,", and welcome to Wikipedia! Thank you for your contributions. I hope you like the place and decide to stay. Here are a few good links for newcomers:\nThe five pillars of Wikipedia\nHow to edit a page\nHelp pages\nTutorial\nHow to write a great article\nManual of Style\nI hope you enjoy editing here and being a Wikipedian! Please sign your name on talk pages using four tildes (~~~~); this will automatically produce your name and the date. If you have any questions, check out Wikipedia:Where to ask a ques...",0,0,0,0,0,0,train,0,1139
144033,06e511d56140e690,"Welcome!\n\nHello, , and welcome to Wikipedia! Thank you for your contributions. I hope you like the place and decide to stay. Here are some pages that you might find helpful:\nThe five pillars of Wikipedia\nHow to edit a page\nHelp pages\nTutorial\nHow to write a great article\nManual of Style\nI hope you enjoy editing here and being a Wikipedian! Please sign your name on talk pages using four tildes (~~~~); this will automatically produce your name and the date. If you need help, check out Wikipedia:Q...",0,0,0,0,0,0,train,0,1139
142539,fa5ea917a9c84811,", and welcome to Wikipedia! Thank you for your contributions. I hope you like the place and decide to stay. Here are some pages that you might find helpful:\nThe five pillars of Wikipedia\nHow to edit a page\nHelp pages\nTutorial\nHow to write a great article\nManual of Style\nI hope you enjoy editing here and being a Wikipedian! Please sign your name on talk pages using four tildes (~~~~); this will automatically produce your name and the date. If you need help, check out Wikipedia:Questions, ask me on...",0,0,0,0,0,0,train,0,1139
141802,f6a48eb98b87d978,"Welcome!\n\nHello, , and welcome to Wikipedia! Thank you for your contributions. I hope you like the place and decide to stay. Here are a few good links for newcomers:\nThe five pillars of Wikipedia\nHow to edit a page\nHelp pages\nTutorial\nHow to write a great article\nManual of Style\nI hope you enjoy editing here and being a Wikipedian! Please sign your name on talk pages using four tildes (~~~~); this will automatically produce your name and the date. If you need help, check out Wikipedia:Questions...",0,0,0,0,0,0,train,0,1139
...,...,...,...,...,...,...,...,...,...,...,...
60584,a22f92cc4b800e6e,"Welcome!\n\nHello, , and welcome to Wikipedia! Thank you for your contributions. I hope you like the place and decide to stay. Here are some pages that you might find helpful:\nThe five pillars of Wikipedia\nTutorial\nHow to edit a page\nHow to write a great article\nManual of Style\nI hope you enjoy editing here and being a Wikipedian! Please sign your name on talk pages using four tildes ~~~~; this will insert your username and the date. If you need help, check out Wikipedia:Questions, ask me on , or ...",0,0,0,0,0,0,train,0,1139
58387,9c4e67ec1d05410c,"Welcome!\n\nHello, , and welcome to Wikipedia! Thank you for your contributions. I hope you like the place and decide to stay. Here are a few good links for newcomers:\nThe five pillars of Wikipedia\nHow to edit a page\nHelp pages\nTutorial\nHow to write a great article\nManual of Style\nI hope you enjoy editing here and being a Wikipedian! Please sign your name on talk pages using four tildes (~~~~); this will automatically produce your name and the date. If 

In [8]:
jc_df.query("group_id == 788").sort_values(by="toxic_flag", ascending=False)

,id,text,toxic,severe_toxic,obscene,threat,insult,identity_hate,layer,toxic_flag,group_id
61011,a35b2e401d106791,"""\nThe Graceful Slick....\nIs non other than an ungraceful dick!86.131.154.230 """,1,0,1,0,1,0,train,3,788
68993,b89a122659669aba,"""\nThe Graceful Slick....\nIs non other than an ungraceful dick!86.138.29.96 """,1,0,1,0,1,0,train,3,788
119714,801f81552ee1ffcd,"""\nThe Graceful Slick....\nIs non other than an ungraceful dick!81.153.48.177 """,1,0,1,0,1,0,train,3,788
122882,91630183539cca60,"""\nThe Graceful Slick....\nIs non other than an ungraceful dick!81.158.100.146 """,1,0,1,0,1,0,train,3,788
27576,48fecdce8be39d02,"""\nThe Graceful Slick....\nIs non other than an ungraceful dick!86.142.21.50 """,1,0,1,0,0,0,train,2,788
62021,a5f1f9a3627f6fbe,"""\nThe Graceful Slick....\nIs non other than an ungraceful dick!86.131.154.225 """,1,0,0,0,0,0,train,1,788
103649,2a8abe0f84f85102,"""\nThe Graceful Slick....\nIs non other than an ungraceful dick!81.131.78.39 """,1,0,0,0,0,0,train,1,788
104269,2df4e82b69123b9d,"""\nThe Graceful Slick....\nIs non other than an ungraceful dick!81.153.49.112 """,1,0,0,0,0,0,train,1,788


In [9]:
jc_df.query("group_id == 481").sort_values(by="toxic_flag", ascending=False)

,id,text,toxic,severe_toxic,obscene,threat,insult,identity_hate,layer,toxic_flag,group_id
69397,b9a1b128936cf730,HELLO NAWLINS \n\nWhy do you like to deflower prepubescent girls? Don’t you know that’s a felony?,1,0,0,0,0,0,train,1,481
85435,e485a952c64de49e,Hello Nawlins \n\nWhy do you like to deflower prepubescent girls? Don’t you know that’s a felony?,1,0,0,0,0,0,train,1,481
61677,a5113609d77ad5bd,"Hello, Nawlins \n\nWhy do you like to deflower prepubescent girls? Don’t you know that’s a felony?",0,0,0,0,0,0,train,0,481
78084,d107dcffa3d24821,"Hello, Nawlins \n\nWhy do you like to deflower prepubescent girls? Don't you know that's a felony?",0,0,0,0,0,0,train,0,481


In [10]:
g = jc_df.groupby(["group_id"]).agg({"toxic":"max","severe_toxic":"max","obscene":"max","threat":"max","insult":"max","identity_hate":"max"}).reset_index()
print(f"jc_df:{jc_df.shape[0]}")
jc_df = jc_df[["group_id", "id", "text"]].drop_duplicates (subset=["group_id"]).reset_index()
print(f"jc_df:{jc_df.shape[0]}")
jc_df = jc_df.merge(g, on="group_id") 
print(f"jc_df:{jc_df.shape[0]}")
jc_df.head()

jc_df:223549
jc_df:220834
jc_df:220834


,index,group_id,id,text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,1150,0000997932d777bf,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",1,0,0,0,0,0
1,1,1151,000103f0d9cfb60f,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0,0,0,0,0,0
2,2,1152,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0,0,0,0,0,0
3,3,1153,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be ...",0,0,0,0,0,0
4,4,1154,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember what page that's on?",0,0,0,0,0,0


In [11]:
g = jc_df.groupby(["group_id"]).agg({"id":"count","toxic":"std","severe_toxic":"std","obscene":"std","threat":"std","insult":"std","identity_hate":"std"}).fillna(0)
g["std"] = g [features].sum(axis=1)
g.query("std != 0").sort_values(by="id", ascending = False)

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate,std
group_id,,,,,,,,


In [12]:
N_FOLDS = 5
mkf = MultilabelStratifiedKFold(n_splits=N_FOLDS, random_state=123, shuffle=True)
y = jc_df [  ["toxic","severe_toxic","obscene","threat","insult","identity_hate"] ]

jc_df["kfold"] = -1
for f, (t_, v_) in enumerate(mkf.split(X=jc_df, y=y)):
    jc_df.loc[v_, 'kfold'] = f
jc_df.to_csv("../processed/jc.csv", index=False)
jc_df.head()

,index,group_id,id,text,toxic,severe_toxic,obscene,threat,insult,identity_hate,kfold
0,0,1150,0000997932d777bf,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",1,0,0,0,0,0,1
1,1,1151,000103f0d9cfb60f,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0,0,0,0,0,0,1
2,2,1152,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0,0,0,0,0,0,1
3,3,1153,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be ...",0,0,0,0,0,0,4
4,4,1154,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember what page that's on?",0,0,0,0,0,0,2


### Build TF-IDF models 

In [13]:
def subsample_majority_class (df, score_column, perc = 1.5):
    min_len = (df[score_column] > 0).sum()
    df_y0_undersample = df[df[score_column] == 0].sample(n=int(min_len*perc),random_state=201)
    df = pd.concat([df[df[score_column] > 0], df_y0_undersample])
                                                
    return df

def cv_tfidf_predict (model_name, df, score_column):

    pred = np.zeros ((df.shape[0]))

    for fold in range(5):
        vec = TfidfVectorizer(analyzer='char_wb', max_df=0.5, min_df=3, ngram_range=(3, 5)) 

        train = df.query ("kfold != @fold")
        
        valid = df.query ("kfold == @fold")

        X_train = vec.fit_transform(train['text'])
        y_train = train[score_column].values

        X_valid = vec.transform(valid['text'])
        y_valid = valid[score_column].values

        model = Ridge (alpha=1.0)
        model.fit(X_train, y_train)    
        pred_train = model.predict ( X_train )
        pred_valid = model.predict ( X_valid )

        pred [valid.index.values] = pred_valid
        
        
        score_train = mean_squared_error(y_train, pred_train, squared=False)
        score_valid = mean_squared_error (y_valid, pred_valid, squared=False )
        print (f"fold:{fold} rmse_train:{score_train:.5f}, rmse_valid:{score_valid:.5f}")
        pickle.dump(model, open(f"../models/{model_name}_{fold}.pkl", 'wb'))
        pickle.dump(vec, open(f"../models/{model_name}_vec_{fold}.pkl", 'wb'))
        


    y_true = df[score_column]
    score_tot = mean_squared_error ( y_true, pred, squared = False)
    print (f"tot score_tot:{score_tot:.5f}")
    return pred


def cv_tfidf_proba (model_name, df, score_column, subsample_perc = 1.5):

    pred = np.zeros ((df.shape[0]))

    for fold in range(5):
        vec = TfidfVectorizer(analyzer='char_wb', max_df=0.5, min_df=3, ngram_range=(3, 5)) 

        train = df.query ("kfold != @fold")
        
        valid = df.query ("kfold == @fold")

        train = subsample_majority_class (train, score_column,  perc = subsample_perc )

        X_train = vec.fit_transform(train['text'])
        y_train = train[score_column].values

        X_valid = vec.transform(valid['text'])
        y_valid = valid[score_column].values

        model = LogisticRegression ()
        model.fit(X_train, y_train)    
        pred_train = model.predict_proba ( X_train )[:,1]
        pred_valid = model.predict_proba ( X_valid )[:,1]

        pred [valid.index.values] = pred_valid
        
        
        auc_train = roc_auc_score(y_train, pred_train)
        auc_valid = roc_auc_score (y_valid, pred_valid )
        print (f"fold:{fold} auc_train:{auc_train:.5f}, auc_valid:{auc_valid:.5f}")
        pickle.dump(model, open(f"../models/{model_name}_{fold}.pkl", 'wb'))
        pickle.dump(vec, open(f"../models/{model_name}_vec_{fold}.pkl", 'wb'))
        


    y_true = df[score_column]
    auc_tot = roc_auc_score ( y_true, pred)
    print (f"tot auc_tot:{auc_tot:.5f}")
    return pred


def tf_idf_feature (texts, feature_name, path, predict_proba = True,  folds=5):
    pred = np.zeros( (len(texts), ) )
    for fold in range(folds):
        model = pickle.load(open(f"{path}/{feature_name}_{fold}.pkl", 'rb'))
        vec = pickle.load(open(f"{path}/{feature_name}_vec_{fold}.pkl", 'rb'))
        X = vec.transform ( texts )
        if predict_proba:
            pred += model.predict_proba ( X )[:,1]
        else:
            pred += model.predict ( X )
            
    return pred/folds




In [14]:
for feat in features:
    print(feat)
    _ = cv_tfidf_proba(model_name=f"jc_tfidf_{feat}", df=jc_df, score_column=feat, subsample_perc = 1.5 )

toxic
fold:0 auc_train:0.98204, auc_valid:0.96648
fold:1 auc_train:0.98144, auc_valid:0.96748
fold:2 auc_train:0.98191, auc_valid:0.96706
fold:3 auc_train:0.98201, auc_valid:0.96866
fold:4 auc_train:0.98206, auc_valid:0.96618
tot auc_tot:0.96717
severe_toxic
fold:0 auc_train:0.99281, auc_valid:0.98506
fold:1 auc_train:0.99298, auc_valid:0.98545
fold:2 auc_train:0.99101, auc_valid:0.98125
fold:3 auc_train:0.99259, auc_valid:0.98577
fold:4 auc_train:0.99242, auc_valid:0.98517
tot auc_tot:0.98454
obscene
fold:0 auc_train:0.99013, auc_valid:0.97965
fold:1 auc_train:0.98990, auc_valid:0.98122
fold:2 auc_train:0.99061, auc_valid:0.98125
fold:3 auc_train:0.99014, auc_valid:0.97870
fold:4 auc_train:0.99033, auc_valid:0.97801
tot auc_tot:0.97977
threat
fold:0 auc_train:0.99397, auc_valid:0.98495
fold:1 auc_train:0.99596, auc_valid:0.97641
fold:2 auc_train:0.99341, auc_valid:0.97771
fold:3 auc_train:0.99272, auc_valid:0.98162
fold:4 auc_train:0.99498, auc_valid:0.97176
tot auc_tot:0.97852
insult

In [15]:
%%time

rud = pd.read_csv("../processed/rud_text.csv")

texts = rud["text"].values
for feat in features:
    feat_name = f"jc_tfidf_{feat}"
    print("rud",feat_name)
    rud[feat_name] = tf_idf_feature(texts, feat_name, path="../models", folds=5)

rud.to_csv("../processed/rud_text_jc_tfidf_fe.csv", index=False)


validation_texts = pd.read_csv ("../processed/validation_texts.csv")
texts = validation_texts["text"].values
for feat in features:
    feat_name = f"jc_tfidf_{feat}"
    print("valid",feat_name)
    validation_texts[feat_name] = tf_idf_feature(texts, feat_name, path="../models", folds=5)

validation_texts.to_csv("../processed/valid_text_jc_tfidf_fe.csv", index=False)

validation_texts.head()

rud jc_tfidf_toxic
rud jc_tfidf_severe_toxic
rud jc_tfidf_obscene
rud jc_tfidf_threat
rud jc_tfidf_insult
rud jc_tfidf_identity_hate
valid jc_tfidf_toxic
valid jc_tfidf_severe_toxic
valid jc_tfidf_obscene
valid jc_tfidf_threat
valid jc_tfidf_insult
valid jc_tfidf_identity_hate
CPU times: user 7min 34s, sys: 3.99 s, total: 7min 38s
Wall time: 7min 38s


,text,jc_tfidf_toxic,jc_tfidf_severe_toxic,jc_tfidf_obscene,jc_tfidf_threat,jc_tfidf_insult,jc_tfidf_identity_hate
0,Whatever gave you the impression that I have the slightest interest in your opinion? Wanker.,0.489397,0.202287,0.201914,0.253144,0.374515,0.212648
1,Jesus god you're fucking retarded \n\nwhat a pathetic piece of shit.,0.999993,0.961022,0.999872,0.720421,0.999881,0.937475
2,"January 4, 2007 \nIf you continue to damage articles, as you have damaged Flute today, this IP will blocked again. FreplySpang",0.195406,0.131218,0.142216,0.235248,0.128284,0.139387
3,"""\n\nDC101 IS HIS FUCKING EMPLOYER!!! IF THAT ISN'T A CREDIBLE SOURCE WHAT IS?????????? If your boss told me you were dead, I would tend to give that some creedence. Maybe reel back the Asperger's a bit and recognize a valid citation for what it is, okay? Notice I said """"valid citation"""", not """"truth"""" because as we both know, Wikipedia doesn't concern itself with truth. Don't be a douche.""",0.760946,0.255794,0.618302,0.197614,0.567498,0.169585
4,"Don't see any attack, unlike the one YOU made on my page, why don't you come up with the evidence before you accuse people!?!???!?!83.67.56.138",0.604014,0.269987,0.356055,0.262937,0.490361,0.208985
